In [1]:
!pip install gradio faster-whisper



  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/3e/a1/277f48516e5284ec0c511d97dc6ae4ba40899e0738d9f2687c1b97786c73/gradio-5.29.0-py3-none-any.whl.metadata
  Using cached gradio-5.29.0-py3-none-any.whl.metadata (16 kB)
  Obtaining dependency information for faster-whisper from https://files.pythonhosted.org/packages/ad/69/28359d152f9e2ec1ff4dff3da47011b6346e9a472f89b409bb13017a7d1f/faster_whisper-1.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for aiofiles<25.0,>=22.0 from https://files.pythonhosted.org/packages/a5/45/30bb92d442636f570cb5651bc661f52b610e2eec3f891a5dc3a4c3667db0/aiofiles-24.1.0-py3-none-any.whl.metadata
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Obtaining dependency information for fastapi<1.0,>=0.115.2 from https://files.pythonhosted.org/packages/50/b3/b51f09c2ba432a576fe63758bddc81f78f0c6309d9e5c10d194313bf021e/fastapi-0.115.12-py3-none-any.whl.metadata
  Using cached fastapi-0.115.12


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import gradio as gr
from faster_whisper import WhisperModel
import random

# Load Whisper model
model = WhisperModel("base")

# Load your Oxford words from local txt file (make sure it's placed in the same folder)
file_path = "oxford_words.txt"
with open(file_path, "r", encoding="utf-8") as file:
    oxford_words = [line.strip() for line in file if line.strip()]

# Select random word function
def select_random_word():
    return random.choice(oxford_words)

# Check pronunciation and give feedback
def check_pronunciation(audio_file, word_to_check):
    try:
        segments, _ = model.transcribe(audio_file)
        result_text = "".join([seg.text for seg in segments]).strip().lower()

        if not result_text:
            return "❌ Could not detect speech. Please try again."

        if word_to_check.lower() in result_text:
            return f"✅ Excellent! AI heard '{result_text}'. Like a native speaker."
        else:
            return f"❌ Poor. AI heard '{result_text}', but expected '{word_to_check}'."

    except Exception as e:
        return f"❌ Error processing audio: {str(e)}"

# Gradio app
with gr.Blocks() as app:
    gr.Markdown("## 🎙 AI Oxford Pronunciation Checker")

    word_display = gr.Textbox(label="🎯 Your Random Word", interactive=False)
    get_word_btn = gr.Button("🎲 Get Random Word")

    # Correct microphone mode (gradio 4+)
    audio_input = gr.Audio(label="🎤 Press and Speak", type="filepath")
    result_output = gr.Textbox(label="AI Pronunciation Feedback")

    get_word_btn.click(fn=select_random_word, inputs=None, outputs=word_display)
    audio_input.change(fn=check_pronunciation, inputs=[audio_input, word_display], outputs=result_output)

app.launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
